In [1]:
import cv2
import numpy as np

In [2]:
def nothing(x):
    # any operation
    pass

In [4]:
cap = cv2.VideoCapture(1)

cv2.namedWindow("Trackbars")
cv2.createTrackbar("thre1", "Trackbars", 40, 255, nothing)
cv2.createTrackbar("thre2", "Trackbars", 40, 255, nothing)
cv2.createTrackbar("thre3", "Trackbars", 100, 120, nothing)
cv2.createTrackbar("thre4", "Trackbars", 0, 1000, nothing)
cv2.createTrackbar("L-H", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L-S", "Trackbars", 90, 255, nothing)
cv2.createTrackbar("L-V", "Trackbars", 50, 255, nothing)
cv2.createTrackbar("U-H", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U-S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U-V", "Trackbars", 255, 255, nothing)

lower = None
upper = None

font = cv2.FONT_HERSHEY_SIMPLEX

while True:
    _, frame = cap.read()
    if frame is None:
        break

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    l_h = cv2.getTrackbarPos("L-H", "Trackbars")
    l_s = cv2.getTrackbarPos("L-S", "Trackbars")
    l_v = cv2.getTrackbarPos("L-V", "Trackbars")
    u_h = cv2.getTrackbarPos("U-H", "Trackbars")
    u_s = cv2.getTrackbarPos("U-S", "Trackbars")
    u_v = cv2.getTrackbarPos("U-V", "Trackbars")

    lower = np.array([l_h, l_s, l_v])
    upper = np.array([u_h, u_s, u_v])

    mask = cv2.inRange(hsv, lower, upper)
    mask = cv2.erode(mask, kernel=np.ones((5, 5), np.uint8), iterations=2)


    thre1 = cv2.getTrackbarPos("thre1", "Trackbars")
    thre2 = cv2.getTrackbarPos("thre2", "Trackbars")

    thre3 = cv2.getTrackbarPos("thre3", "Trackbars")
    thre4 = cv2.getTrackbarPos("thre4", "Trackbars")

    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    canny_frame = cv2.Canny(blurred_frame, thre1, thre2)
    canny_frame = cv2.dilate(canny_frame, kernel=np.ones((3, 3), np.uint8), iterations=1)

    mask1 = np.ones(frame.shape, dtype=np.uint8)
    contours1, _ = cv2.findContours(canny_frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt1 in contours1:
        x, y, w, h = cv2.boundingRect(cnt1)
        # # Create a mask from a boundingRect in openCV
        if w>thre3 and h>thre3 and w<2.1*thre3 and h<2.1*thre3:
            cv2.rectangle(mask1, (x, y), (x + w, y + h), (255, 255, 255), -1)
            mask1 = cv2.erode(mask1, kernel=np.ones((5, 5), np.uint8), iterations=2)
            mask1 = cv2.dilate(mask1, kernel=np.ones((5, 5), np.uint8), iterations=1)


    # Contours detection
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)
        x = approx.ravel()[0]
        y = approx.ravel()[1]

        if area > thre4:
            cv2.drawContours(frame, [approx], 0, (255, 255, 255), 2)
            cv2.fillPoly(frame, [approx], (255, 255, 255))

            if len(approx) == 4:
                cv2.putText(frame, "Rectangle", (x, y), font, 0.6, (255, 255, 255), 2)

    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask)
    cv2.imshow("blurred_frame", blurred_frame)
    cv2.imshow("Edge", canny_frame)
    cv2.imshow("Mask1", mask1)
    if cv2.waitKey(30) == ord('q'):
        break

print(lower, upper)
cv2.destroyAllWindows()
cap.release()

[ 0 90 50] [255 255 255]
